In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    i = 0
    for batch in dataloader:
        print('Batch number ', i)
        i += 1
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs, training=True)
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader)}")

print("Training complete.")

In [ ]:
# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in dataloader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test dataset: {100 * correct / total:.2f}%')

In [ ]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs):

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        print(f"Validation Loss: {val_loss/len(valid_loader)}")

    return model

In [ ]:
learning_rate = 1e-3
num_epochs = 10

model = Model(num_frames=384, num_landmarks=118, num_classes=250, dim=192).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Parameters
num_folds = 5
batch_size = 64

# Split the dataset indices for k-fold cross-validation
indices = np.arange(len(dataset))
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [ ]:
def get_dataloaders(dataset, num_folds, batch_size, test_size_ratio=0.1):
    indices = np.arange(len(dataset))
    test_size = int(test_size_ratio * len(dataset))
    test_indices = np.random.choice(indices, test_size, replace=False)

    train_val_indices = np.setdiff1d(indices, test_indices)
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    fold_data = []

    for train_idx, val_idx in kf.split(train_val_indices):
        train_idx = train_val_indices[train_idx]
        val_idx = train_val_indices[val_idx]
        
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        
        fold_data.append((train_loader, val_loader))
    
    test_subset = Subset(dataset, test_indices)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)
    
    return fold_data, test_loader

In [ ]:
fold_data, test_loader = get_dataloaders(dataset, num_folds, batch_size)

for fold, (train_loader, val_loader) in enumerate(fold_data):
    print(f"Fold {fold + 1}:")
    print(f"Train loader has {len(train_loader.dataset)} samples")
    print(f"Val loader has {len(val_loader.dataset)} samples")
print(f"Test loader has {len(test_loader.dataset)} samples")

In [ ]:
def train_one_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0

    i = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        print('-'*50)
        print('train iteration', i)
        i += 1
        print('inputs ', inputs.shape)
        print('outputs ', outputs.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
   
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

def validate_one_epoch(model, dataloader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct_predictions = 0

    i = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            print('-'*50)
            print('val iteration', i)
            i += 1
            print('inputs ', inputs.shape)
            print('outputs ', outputs.shape)
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = correct_predictions.float() / len(dataloader.dataset)  # Ensure this is float32
    return epoch_loss, accuracy


In [ ]:
def train_k_folds(model_class, dataset, num_folds, batch_size, num_epochs, lr, test_size_ratio=0.1, device='mps'):
    fold_data, test_loader = get_dataloaders(dataset, num_folds, batch_size, test_size_ratio)

    for fold, (train_loader, val_loader) in enumerate(fold_data):
        print(f"Training fold {fold + 1}...")

        model = model_class().to(device)
        optimizer = Adam(model.parameters(), lr=lr)
        criterion = torch.nn.CrossEntropyLoss()

        for epoch in range(num_epochs):
            print('='*50)
            train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
            print('='*50)
            print('train_loss', train_loss)
            print('='*50)
            val_loss, val_accuracy = validate_one_epoch(model, val_loader, criterion, device)
            print('='*50)
            
            print(f"Fold {fold + 1} Epoch {epoch + 1}/{num_epochs}")
            print(f"Train Loss: {train_loss:.4f}")
            print(f"Validation Loss: {val_loss:.4f}")
            print(f"Validation Accuracy: {val_accuracy:.4f}")

    return model, test_loader

learning_rate = 1e-3
num_epochs = 1

print('='*50)

model = Model(num_frames=384, num_landmarks=118, num_classes=250, dim=192).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

trained_model, test_loader = train_k_folds(Model, dataset, num_folds, batch_size, num_epochs, learning_rate)
print('='*50)

In [ ]:
def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0

    i = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            print('-' * 50)
            print('test iteration', i)
            i += 1
            print('inputs ', inputs.shape)
            print('outputs ', outputs.shape)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels.data).float()  # Ensure float32

    test_loss = running_loss / len(test_loader.dataset)
    accuracy = correct_predictions / len(test_loader.dataset)  # Remain in float32
    return test_loss, accuracy

print('='*50)
# Evaluate the final model on the test set
criterion = torch.nn.CrossEntropyLoss()
test_loss, test_accuracy = evaluate(trained_model, test_loader, criterion, device='mps')

print('='*50)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print('='*50)